# IMPORTS

In [0]:
#INSTALLATIONS
!mkdir models
!pip install -q keras
import keras
!pip install -q tqdm
import tqdm
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

#LE FUTUR WESH
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#CLASSIQUES
from google.colab import files, auth
from oauth2client.client import GoogleCredentials
import numpy as np
np.random.seed(1000)
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
from tqdm import tqdm
import matplotlib.pyplot as plt
import timeit
import sklearn.metrics as sklm
import copy
import types as python_types
import warnings
import pickle

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('ATTENTION ! PAS DE GPU !')
else:
  print('Found GPU at: {}'.format(device_name))

#KERAS
import keras
from keras import activations, initializers, regularizers, constraints,metrics
from keras.legacy import interfaces
from keras.engine import InputSpec, Layer
from keras.layers import Input,BatchNormalization
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D, MaxPooling2D
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import Adam, Adagrad
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.utils import np_utils
from keras.utils.generic_utils import func_dump, func_load, deserialize_keras_object

# DRIVE AUTHENTIFICATION
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# CONSTANTES

In [0]:
# QUELQUES VARIABLES

adagrad = Adagrad(lr=0.001, epsilon=None, decay=0.0) #c'est celui-là qui est utilisé pour tous les modèles
adam = Adam(lr=0.0002, beta_1=0.5)

randomVectorSize = 100

d_losses = []
g_losses = []
c_losses = []
r_losses = []

# Nombre de learning steps par batch du DISCRIMINATOR
nb_step_discriminator = 1

# Seuil de score du DISCRIMINATOR en deça duquel on considere un exemple fake comme exemple classe 1 du REJECTOR
seuil_rejet_RejectionTraining = 0.45

# Seuil de score du DISCRIMINATOR au-delà duquel on considere un exemple fake comme exemple classe 0 du REJECTOR
seuil_accept_RejectionTraining = 0.55

# Nombre maximum d'exemples dans allGeneratedSamples
max_GeneratedSamples = 20000

# Permutation aléatoire des exemples dans allGeneratedSamples lorsque max_GeneratedSamples exemples est atteint
randomPermut_GeneratedSamples = True

# Tri des exemples dans allGeneratedSamples en fonction de leurs scores
sort_GeneratedSamples = False

# Cible des exemples negatifs 
target_neg = int(0)

# Cible des exemples positifs 
target_pos = int(1)

#Seuil à appliquer pour les prédictions du REJECTOR
seuil_rejector = 0.5

#Seuil à appliquer aux sorties du REJECTOR lors de l'entraînement du CLASSIFIER
seuil_rejector_to_train_classifier = 0.0

# Taux de decroisance de la variance des rbf units
Decrease_rate_alpha = 1.1  

# Utilisation des données perturbées
usePerturbatedData = False

# Nombre de classes dans MNIST
nb_classes = 10

# Choix des classes à traiter par le réseau
classes_train = np.random.choice(range(nb_classes), size=3, replace=False)
classes_train =[0, 4, 7]
classes_not_train = [i for i in range(nb_classes) if i not in classes_train]

In [7]:
# IMPORTATION MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32).reshape((X_train.shape[0],28,28,1)) - 127.5)/127.5
Y_train = np_utils.to_categorical(y_train, nb_classes)
#Y_train = Y_train * 0.9
X_test = (X_test.astype(np.float32).reshape((X_test.shape[0],28,28,1)) - 127.5)/127.5
Y_test = np_utils.to_categorical(y_test, nb_classes)
#Y_test = Y_test * 0.9

indices = np.where(np.isin(y_train, classes_train))
X_train_restricted = X_train[indices,:]
X_train_restricted = X_train_restricted.reshape(X_train_restricted.shape[1],28,28,1)
Y_train_restricted = Y_train[indices,:]
Y_train_restricted = Y_train_restricted.reshape(Y_train_restricted.shape[1],10)

indices = np.where(np.isin(y_train, classes_not_train))
X_train_out = X_train[indices,:]
X_train_out = X_train_out.reshape(X_train_out.shape[1],28,28,1)
Y_train_out = Y_train[indices,:]
Y_train_out = Y_train_out.reshape(Y_train_out.shape[1],10)

indices = np.where(np.isin(y_test, classes_train))
X_test_restricted = X_test[indices,:]
X_test_restricted = X_test_restricted.reshape(X_test_restricted.shape[1],28,28,1)
Y_test_restricted = Y_test[indices,:]
Y_test_restricted = Y_test_restricted.reshape(Y_test_restricted.shape[1],10)

indices = np.where(np.isin(y_test, classes_not_train))
X_test_out = X_test[indices,:]
X_test_out = X_test_out.reshape(X_test_out.shape[1],28,28,1)
Y_test_out = Y_test[indices,:]
Y_test_out = Y_test_out.reshape(Y_test_out.shape[1],10)

11493376/11490434 [==============================] - 3s 0us/step


# FONCTION UTILITAIRES

In [0]:
#@title CALCUL DE PERFORMANCES
###________FONCTIONS POUR CALCULER LES PERFORMANCES________###

#MOYENNE DES PREDICTIONS CORRECTES
def mean_pred_pos(y_true, y_pred):
  good_examples = np.where(y_true[:,0] ==0)
  return K.mean(y_pred[good_examples])
  
#MOYENNE DES PREDICTIONS FAUSSES
def mean_pred_neg(y_true, y_pred):
  good_examples = np.where(y_true[:,0] ==1)
  return K.mean(y_pred)

#RECALL RATE (vrais positifs / (vrais positifs + faux négatifs))
def recall(y_true, y_pred):
  res  = K.eval(tf.multiply (y_true, y_pred))
  true_positives = np.sum(np.round(np.clip(res, 0, 1)))
  possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

#PRECISION (vrais positifs / (vrais positifs + faux positifs))
def precision(y_true, y_pred):
  res  = K.eval(tf.multiply (y_true, y_pred))
  true_positives = np.sum(np.round(np.clip(res, 0, 1)))
  predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

#F1 = moyenne harmonique du recall rate et de la précision
def f1(y_true, y_pred):
    precision_f1 = precision(y_true, y_pred)
    recall_f1 = recall(y_true, y_pred)
    return 2*((precision_f1*recall_f1)/(precision_f1+recall_f1))

#AREA UNDER CURVE de la courbe Faux positifs/Vrais positifs...
def auc(y_true, y_pred):  
  fpr, tpr, thresholds = sklm.roc_curve(y_true, y_pred, pos_label=1)
  return sklm.auc(fpr, tpr)

In [0]:
#@title COUCHES RBF
###________TROIS CLASSES POUR IMPLEMENTER DES COUCHES RBF________###

class RBFLayer(Layer):
    def __init__(self, alpha=1000.0, alpha_initializer='zeros',
                 alpha_regularizer=None,
                 alpha_constraint=None, 
                 **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.supports_masking = True
        self.supports_masking = True
        self.alpha_initializer = keras.initializers.Constant(value=1000.0)
        self.alpha_regularizer = regularizers.get(alpha_regularizer)
        self.alpha_constraint = constraints.get(alpha_constraint)
        self.alpha = self.add_weight(shape=(1,),
                                     name='alpha',
                                     initializer=self.alpha_initializer,
                                     regularizer=self.alpha_regularizer,
                                     constraint=self.alpha_constraint)
        self.trainable = False

    def call(self, inputs):
        y = - inputs / self.alpha[0]
        y = keras.backend.exp(y)
        return y

    def get_config(self):
        config = {
            'alpha_initializer': initializers.serialize(self.alpha_initializer),
            'alpha_regularizer': regularizers.serialize(self.alpha_regularizer),
            'alpha_constraint': constraints.serialize(self.alpha_constraint),
            'shared_axes': self.shared_axes
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

class RBF(Layer):
    @interfaces.legacy_dense_support
    def __init__(self, units,
                 activation=None,
                 use_bias=False,
                 kernel_initializer='TruncatedNormal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 sigma = 1000.0,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(RBF, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.sigma = sigma
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs):
        norm_x = K.transpose(K.sum(inputs * inputs , axis=1))
        norm_w = K.sum(self.kernel *self.kernel , axis=0)
        norm_x = keras.backend.expand_dims(norm_x, axis=-1)
        prod_scal = -2 * K.dot(inputs, self.kernel)
        print ("Shapes : ", norm_x.shape, norm_w.shape, prod_scal.shape) 
        a = tf.add(norm_x, prod_scal) # -2 *  prod_scal)
        y = tf.add(a, norm_w )
        y = - y / self.sigma
        y = keras.backend.exp(y)
        return y

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(RBF, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class RBF2(Layer):
    @interfaces.legacy_dense_support
    def __init__(self, units,
                 activation=None,
                 use_bias=False,
                 kernel_initializer='TruncatedNormal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(RBF2, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs):
        norm_x = K.transpose(K.sum(inputs * inputs , axis=1))
        norm_w = K.sum(self.kernel *self.kernel , axis=0)
        norm_x = keras.backend.expand_dims(norm_x, axis=-1)
        prod_scal = -2 * K.dot(inputs, self.kernel)
        print ("Shapes : ", norm_x.shape, norm_w.shape, prod_scal.shape) 
        a = tf.add(norm_x, prod_scal) # -2 *  prod_scal)
        y = tf.add(a, norm_w )
        return y

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(RBF2, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
#@title AFFICHAGE
###________FONCTIONS D'AFFICHAGE________###

# AFFICHAGE DES LOSS DU GAN
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(d_losses, label='Discriminitive loss')
    plt.plot(g_losses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_e%d.png' % epoch)
    uploaded = drive.CreateFile({'title': 'loss_e%d.png' % epoch})
    uploaded.SetContentFile('loss_e%d.png' % epoch)
    uploaded.Upload()

# EXEMPLES D'IMAGES GENEREES
def plotGeneratedImages(epoch, examples=100):
    noise = np.random.normal(0, 1, size=[examples, randomVectorSize])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)
    Xg = generatedImages[:8]
    nom_file = "gan_generated_image_epoch_" +str(epoch) + ".png"
    print (nom_file)
    np.save(nom_file, Xg)
    #files.download(nom_file)

# EXEMPLES D'IMAGES GENEREES 2
def plotGeneratedImages2(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomVectorSize])
    generatedImages = generator.predict(noise)
    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)
    files.download('gan_generated_image_epoch_%d.png' % epoch)    

# AFFICHE LES IMAGES (mettez-en 100 ou moins svp sinon je sais pas ce que ça va faire)
def plotImages(images, dim=(10, 10), figsize=(10, 10)):
    plt.figure(figsize=figsize)
    for i in range(images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(images[i, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_perturbated_images.png')
    files.download('gan_perturbated_images.png')  

In [0]:
#@title MISE EN FORME DE DONNEES
###________FONCTIONS DE MISE EN FORME DES DONNEES________###

# MET EN FORME LES LABELS ET LES DONNEES POUR LE TEST DU CLASSIFIEUR SUR LES DONNEES NON REJETEES
def data_evaluation_classifier(X_real, y_real):
  y_dgan = rejector.predict(X_real)
  good_examples = np.where(y_dgan[:,0] < seuil_rejector)
  X_disc = X_real[good_examples]
  y_disc = y_real[good_examples]
  weights = np.ones((y_real.shape[0],1))
  return X_disc, y_disc

# MET EN FORME LES LABELS POUR DES ENTREES X TOUTES VRAIES
def data_evaluation_rejector_true(X):
    return X, np.ones((X.shape[0],1))

# MET EN FORME LES LABELS ET LES DONNEES POUR LES IMAGES GENEREES
def data_evaluation_rejector_fake(X_fake, filtre):
  if filtre:
    y_reject = discriminator.predict(X_fake)
    good_examples = np.where(y_reject[:,0] < seuil_rejet_RejectionTraining)
    X_disc = X_fake[good_examples]
    y_disc = np.zeros((X_disc.shape[0],1))
  else:
    X_disc = X_fake[:]
    y_disc = np.zeros((X_disc.shape[0],1))
  return X_disc, y_disc

# MODIFIE DES IMAGES EN FAISANT DU COPIER/COLLER (à reprendre en modifiant)
def imageBatch_mix(batchsize):
  lesindices =  np.random.randint(0, X_train_restricted.shape[0], size=batchsize)
  imageBatch_to_perturb = X_train_restricted[lesindices,:]
  imageBatch_to_perturb = imageBatch_to_perturb.reshape((imageBatch_to_perturb.shape[0], X_train.shape[1]*X_train.shape[2]))
  data_dim = imageBatch_to_perturb.shape[1]
  X_perturbated = imageBatch_to_perturb
  rate1 = 0.9
  seuil1 = int(rate1* (float(data_dim)))
  seuil1 = np.random.randint(0, high=seuil1)
  X_perturbated[:,:seuil1] = imageBatch_to_perturb[:,:seuil1]
  rate2 = 0.9
  seuil2 = int(rate2* (float(data_dim)))
  seuil2 = np.random.randint(0, high=seuil2)
  seuil2 = min(seuil1+seuil2, data_dim)
  p = np.random.permutation(batchsize)
  X_perturbated[:,seuil1:seuil2] = imageBatch_to_perturb[p,seuil1:seuil2]
  p = np.random.permutation(batchsize)  
  X_perturbated[:,seuil2:] = imageBatch_to_perturb[p,seuil2:]
  X_perturbated = X_perturbated.reshape((imageBatch_to_perturb.shape[0], 28, 28, 1))
  return X_perturbated

# RENVOIE UN BATCH D'ENTRAINEMENT POUR LE REJECTOR
def data_train_rejector(X_real, y_real, X_fake, X_perturbated):
    batchSize = X_real.shape[0]
    y_fake_reject = discriminator.predict(X_fake)
    good_examples = np.where(y_fake_reject[:,0] < seuil_rejet_RejectionTraining )
    bad_examples = np.where(y_fake_reject[:,0] > seuil_accept_RejectionTraining )
    X_fake_good = X_fake[good_examples[0]]
    X_fake_good= X_fake_good[:min(X_fake_good.shape[0],batchSize)]
    y_fake_good = np.zeros((y_fake_reject[good_examples[0]].shape[0],1))
    y_fake_good= y_fake_good[:min(X_fake_good.shape[0],batchSize)]
    y_fake_good[:,] = 0
    X_fake_bad = X_fake[bad_examples[0]]
    X_fake_bad = X_fake_bad[:min(X_fake_bad.shape[0],batchSize)]
    y_fake_bad = np.zeros((y_fake_reject[bad_examples[0]].shape[0],1))
    y_fake_bad = y_fake_bad[:min(X_fake_bad.shape[0],batchSize)]
    y_fake_bad[:,] = 1
    y_real_reject = np.ones((X_real.shape[0],1))
    y_perturbated = np.zeros((X_perturbated.shape[0],1))
    if usePerturbatedData:
      X = np.concatenate([X_real, X_fake_good, X_fake_bad, X_perturbated])
      y_rejection = np.concatenate([y_real_reject, y_fake_good, y_fake_bad, y_perturbated])
    else:
      X = np.concatenate([X_real, X_fake_good, X_fake_bad])   
      y_rejection = np.concatenate([y_real_reject, y_fake_good, y_fake_bad])
    weights = np.ones((y_rejection.shape[0],1))
    return X, y_rejection, weights

# RENVOIE LES ELEMENTS DE X_REAL NON REJETES PAR LE REJECTOR (à modifier pour inclure les données perturbées)
def data_train_classifier(X_real, y_real, X_perturbated):
    y_dgan = rejector.predict(X_real)
    good_examples = np.where(y_dgan[:,0] > seuil_rejector_to_train_classifier)
    X_disc = X_real[good_examples]
    y_disc = y_real[good_examples]
    weights = np.ones((y_disc.shape[0],1))
    """
    y_perturbated = np.zeros((X_perturbated.shape[0],y_disc.shape[1]))
    X = np.concatenate([X_disc, X_perturbated])
    y = np.concatenate([y_disc, y_perturbated])
    weights = np.ones((y.shape[0],1))
    """
    return X_disc, y_disc, weights

#APPLIQUE UNE PERMUTATION DES IMAGES GENEREES (optionnel) PUIS FILTRE LES IMAGES EN FONCTION DE LEUR REPONSE AU DISCRIMINATEUR
def filtrage(allgeneratedImages, nb_generated_max = max_GeneratedSamples):
    if randomPermut_GeneratedSamples:
      p = np.random.permutation(range(allgeneratedImages.shape[0]))
      allgeneratedImages = allgeneratedImages[p]
    y_fake_reject = discriminator.predict(allgeneratedImages)
    good_examples = np.where(y_fake_reject[:,0] < seuil_rejet_RejectionTraining )
    X_temp = allgeneratedImages[good_examples]
    if (X_temp.shape[0]> nb_generated_max):
        return X_temp[:nb_generated_max]
    else:
        if (X_temp.shape[0]==0):
            return allgeneratedImages[:min(allgeneratedImages.shape[0],nb_generated_max)]
        else:
            return X_temp          
          
# SAUVEGARDER LES MODELES
def saveModels(epoch):
  generator.save('generator_e%d' % epoch)
  discriminator.save('discriminator_e%d' % epoch)

# TELECHARGER LES MODELES
def downloadModels(epoch):
  uploaded = drive.CreateFile({'title': 'generator_e%d' % epoch})
  uploaded.SetContentFile('generator_e%d' % epoch)
  uploaded.Upload()
  uploaded = drive.CreateFile({'title': 'discriminator_e%d' % epoch})
  uploaded.SetContentFile('discriminator_e%d' % epoch)
  uploaded.Upload()

# UPLOADER LES MODELES
def uploadModels(epoch):
  genID = drive.ListFile({'title':'generator_e%d' % epoch}).GetList()[0]['id']
  gen = drive.CreateFile({'id':genID})
  gen.GetContentFile('generator_e%d' % epoch)
  generator = keras.load_model('generator_e%d' % epoch)
  discID = drive.ListFile({'title':'discriminator_e%d' % epoch}).GetList()[0]['id']
  disc = drive.CreateFile({'id':discID})
  disc.GetContentFile('discriminator_e%d' % epoch)
  discriminator = keras.load_model('discriminator_e%d' % epoch)
  return generator, discriminator

#Je sais pas encore ni ce que c'est ni à quoi ça sert, ça ne semble pas utilisé
def toutes_data_evaluation_classifier(X, y):
  y_disc = np.concatenate((np.zeros((y.shape[0],1)), y ), axis=1)
  return X, y_disc

# Ne semble pas utilisé
def imageBatch_neg(batchsize):
  lesindices =  np.random.randint(0, X_train.shape[0], size=batchsize*3)
  imageBatch_to_perturb = X_train[lesindices,:]
  X_perturbated = imageBatch_to_perturb[:batchsize] + imageBatch_to_perturb[batchsize:2*batchsize]- imageBatch_to_perturb[2*batchsize:3*batchsize]
  #X_perturbated = (imageBatch_to_perturb[:batchsize] + imageBatch_to_perturb[batchsize:2*batchsize])/2
  return X_perturbated

In [0]:
#@title SEUILLAGES

#Application d'un seuil en sortie du classifier sur la probabilité la plus grande
def test_model(model,seuil,x_test,y_test):
  print("Seuillage sur la probabilité max :")
  print("Seuil = "+str(seuil))
  predictions=model.predict(x_test)
  indices=[]
  for i in range(len(x_test)):
    #print(max(predictions[i]))
    if max(predictions[i])>seuil:
      indices.append(i)
  new_x_test=[]
  new_y_test=[]
  if len(indices)==0:
    print("Toutes les images sont rejetées...")
    return([len(x_test),1])
  for indice in indices:
    new_x_test.append(x_test[indice])
    new_y_test.append(y_test[indice])
  new_x_test=np.array(new_x_test)
  new_y_test=np.array(new_y_test)
  new_score=model.evaluate(new_x_test,new_y_test,batch_size=16)
  nb_total=len(x_test)
  nb_conservees=len(new_x_test)
  nb_rejetees=nb_total-nb_conservees
  rapport_conservees=nb_conservees/nb_total
  print("Nombre d'images conservées : "+str(nb_conservees)+"/"+str(nb_total)+" = "+str(100*rapport_conservees)+"%")
  print("Nombre d'images rejetées : "+str(nb_rejetees))
  print("Score avec les images conservées : ",new_score)
  return([nb_rejetees,new_score[1]])


#Calcul de l'entropie
def entropie(array):
  H=0
  for j in range(len(array)):
    H+=-1*array[j]*np.log2(array[j])
  return(H)


#Application d'un seuil sur l'entropie de la sortie du classifier
def test_model_entropie(model,seuil,x_test,y_test):
  print("Seuillage sur l'entropie : ")
  print("Seuil = "+str(seuil))
  predictions=model.predict(x_test)
  indices=[]
  for i in range(len(x_test)):
    #print(entropie(predictions[i]))
    if entropie(predictions[i])<seuil:
      indices.append(i)
  new_x_test=[]
  new_y_test=[]
  if len(indices)==0:
    print("Toutes les images sont rejetées...")
    return([len(x_test),1])
  for indice in indices:   
    new_x_test.append(x_test[indice])
    new_y_test.append(y_test[indice])
  new_x_test=np.array(new_x_test) 
  new_y_test=np.array(new_y_test)
  new_score=model.evaluate(new_x_test,new_y_test,batch_size=16)
  nb_total=len(x_test)
  nb_conservees=len(new_x_test)
  nb_rejetees=nb_total-nb_conservees
  rapport_conservees=nb_conservees/nb_total
  print("Nombre d'images conservées : "+str(nb_conservees)+"/"+str(nb_total)+" = "+str(100*rapport_conservees)+"%")
  print("Nombre d'images rejetées : "+str(nb_rejetees))
  print("Score avec les images conservées : ",new_score)
  return([nb_rejetees,new_score[1]])


# MODELES

In [13]:
# Generator

g_input = Input(shape=(randomVectorSize,),name="g_input")
x = Dense(7*7*128, kernel_initializer=initializers.RandomNormal(stddev=0.02))  (g_input)
x = LeakyReLU(0.2)                                                             (x)
x = Reshape((7,7,128))                                                         (x)
x = UpSampling2D(size=(2, 2))                                                  (x)
x = Conv2D(64, kernel_size=(5, 5), padding='same')                             (x)
x = LeakyReLU(0.2)                                                             (x)
x = UpSampling2D(size=(2, 2))                                                  (x)
g_prediction = Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh')(x)

generator = Model(input = g_input, output = g_prediction)
generator.compile(optimizer=adagrad, loss='binary_crossentropy')
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_input (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 6272)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        204864    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)        0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("g_..., outputs=Tensor("co...)`
  if sys.path[0] == '':


In [14]:
# Discriminator

d_input = Input(shape=(28,28,1),name="d_input")
x = Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', kernel_initializer=initializers.RandomNormal(stddev=0.02)) (d_input)
x = LeakyReLU(0.2)                                                 (x)
x = Dropout(0.3)                                                   (x)
x = Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same')(x)
x = LeakyReLU(0.2)                                                 (x)
x = Dropout(0.3)                                                   (x)
x = Flatten()                                                      (x)
d_prediction = Dense(1, activation='sigmoid', name='d_output')     (x)

discriminator = Model(input = d_input, output = d_prediction)
discriminator.compile(optimizer=adagrad, loss='binary_crossentropy')
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
d_input (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 128)         0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("d_..., outputs=Tensor("d_...)`
  if sys.path[0] == '':


In [15]:
# GAN

discriminator.trainable = False
gan_input = Input(shape=(randomVectorSize,))
x = generator (gan_input)
gan_prediction = discriminator (x)

gan = Model(input = gan_input, output = gan_prediction)
gan.compile(optimizer=adagrad, loss='binary_crossentropy')
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_1 (Model)              (None, 28, 28, 1)         839937    
_________________________________________________________________
model_2 (Model)              (None, 1)                 212865    
Total params: 1,052,802
Trainable params: 839,937
Non-trainable params: 212,865
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`
  import sys


In [16]:
# Classifier

c_input = Input(shape=(28,28,1), name='c_input')
x =  Conv2D(64, kernel_size=(3, 3), activation='relu') (c_input)
x = MaxPooling2D(pool_size=(2, 2))                     (x)
x = Conv2D(128, (3, 3), activation='relu')             (x)
x = MaxPooling2D(pool_size=(2, 2))                     (x)
x = Dropout(0.25)                                      (x)
x = Flatten()                                          (x)
x = Dense(128, activation='relu')                      (x)
#x = Dropout(0.5)                                       (x)
c_prediction = Dense(10, activation='softmax', name='c_output') (x)

classifier = Model(input = c_input, output = c_prediction)
classifier.compile(optimizer=adagrad, loss= 'binary_crossentropy', metrics=[metrics.categorical_accuracy])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c_input (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("c_..., outputs=Tensor("c_...)`
  del sys.path[0]


In [17]:
# Rejector

r_input = Input(shape=(28,28,1), name='r_input')
x =  Conv2D(64, kernel_size=(3, 3), activation='relu')(r_input)
x = MaxPooling2D(pool_size=(3, 3))                    (x)
x = Conv2D(128, (3, 3), activation='relu')            (x)
x = MaxPooling2D(pool_size=(3, 3))                    (x)
x = Dropout(0.25)                                     (x)
x = Flatten()                                         (x)
#x = RBF(128, sigma=1000.0)                            (x)
x = Dense(128, activation='relu')                     (x)
x = RBF2(300)                                         (x)
x = RBFLayer(alpha=2000.0)                            (x)
#x = BatchNormalization()                              (x)
#x = Dropout(0.5)                                      (x)
r_prediction = Dense(1, activation='tanh', name='r_output', use_bias=False) (x)

rejector = Model(input = r_input, output = r_prediction)
rejector.compile(optimizer=adagrad, loss= 'mse', metrics=['acc'])  #  metrics=['acc', f1, precision, recall])
rejector.summary()

Shapes :  (?, 1) (300,) (?, 300)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
r_input (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 128)         73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("r_..., outputs=Tensor("r_...)`


# TRAININGS ET BASES DE DONNEES

In [0]:
#@title ENTRAINEMENT DE TOUS LES RESEAUX
def train_all(X_train, Y_train, X_test_in, Y_test_in, X_test_out, Y_test_out, epochs=100, batchSize=128):
  #INITIALISATION...
  batchCount = int(X_train.shape[0] / batchSize)
  print ('Epochs:', epochs)
  print ('Batch size:', batchSize)
  print ('Batches per epoch:', batchCount)
  noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
  allgeneratedImages = generator.predict(noise)

  for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    rloss=0
    closs=0
    dloss=0
    gloss=0
    
    # ENTRAINEMENT RESEAUX
    for _ in tqdm(range(batchCount)):
      # DISCRIMINATOR
      noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
      generatedImages = generator.predict(noise)
      imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
      X = np.concatenate([imageBatch, generatedImages])
      yDis = np.zeros(2*batchSize)
      yDis[:batchSize] = 0.9
      dloss += discriminator.train_on_batch(X, yDis)

      # GENERATOR
      noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
      yGen = np.ones(batchSize)
      discriminator.trainable = False
      gloss += gan.train_on_batch(noise, yGen)
      
      # RECUPERATION DES IMAGES GENEREES PERTINENTES
      allgeneratedImages = np.concatenate ([allgeneratedImages, generatedImages])
      allgeneratedImages = filtrage(allgeneratedImages)

      # CLASSIFIER
      indices = np.random.randint(0, X_train.shape[0], size=2*batchSize)
      imageBatch = X_train[indices,:]
      imageBatch_labels = Y_train[indices,:]
      imageBatch_perturbated = imageBatch_mix(batchSize)
      X_disc, y_disc, weights = data_train_classifier(imageBatch, imageBatch_labels, imageBatch_perturbated)#pour l'instant cette fonction ne prend pas en compte les données perturbées => à coder
      if (X_disc.shape[0]> 10):
        scores = classifier.train_on_batch(X_disc, y_disc)
        closs += scores[0]

      # REJECTOR
      X_disc, y_disc, weights = data_train_rejector(imageBatch, imageBatch_labels, allgeneratedImages, imageBatch_perturbated)
      scores = rejector.train_on_batch(X_disc, y_disc)
      rloss += scores[0]

    # TESTS ET EVALUATIONS
    d_losses.append(dloss)
    g_losses.append(gloss)
    r_losses.append(rloss)
    c_losses.append(closs)

    X_disc, y_disc = data_evaluation_rejector_true(X_train)
    r_train_in = rejector.evaluate(X_disc, y_disc)
    X_disc, y_disc = data_evaluation_rejector_true(X_test_in)
    r_test_in = rejector.evaluate(X_disc, y_disc)
    X_disc, y_disc = data_evaluation_rejector_fake(X_test_out, filtre=False)
    r_test_out = rejector.evaluate(X_disc, y_disc)
    X_disc, y_disc = data_evaluation_rejector_fake(allgeneratedImages, filtre=True)
    r_generated_filtre = rejector.evaluate(X_disc, y_disc)
    X_disc, y_disc = data_evaluation_rejector_fake(allgeneratedImages, filtre=False)
    r_generated = rejector.evaluate(X_disc, y_disc)
    X_disc, y_disc = data_evaluation_classifier(X_test,Y_test)
    a_eval_pred = classifier.evaluate(X_disc, y_disc)
    a_pred_in = classifier.evaluate(X_test_in, Y_test_in)
    a_pred_out = classifier.evaluate(X_test_out, Y_test_out)

    print ("Nombre d'exemples générés : ", allgeneratedImages.shape[0])
    print ("Score du REJECTOR sur les données d'entraînement de MNIST : ", r_train_in)
    print ("Score de REJECTOR sur les données générées par le GAN : ", r_generated)
    print ("Score du REJECTOR sur les données générées par le GAN filtrées : ", r_generated_filtre)
    print ("Score du REJECTOR sur les données de test de MNIST des classes attendues : ", r_test_in)
    print ("Score du REJECTOR sur les données de test de MNIST des classes non attendues : ", r_test_out)
    print ("Score du CLASSIFIER sur les données de test de MNIST non rejetées : ", a_eval_pred)
    print ("Score du CLASSIFIER sur les données de test de MNIST des classes attendues : ", a_pred_in)
    print ("Score du CLASSIFIER sur les données de test de MNIST des classes non attendues : ", a_pred_out)

    Xin,Yin = data_evaluation_rejector_true(X_test_in)
    y_pred_in = rejector.predict(Xin)
    Xout,Yout = data_evaluation_rejector_fake(X_test_out, filtre=False)
    y_pred_out = rejector.predict(Xout)
    Ytrue = np.concatenate ((Yout,Yin))
    Ypred = np.concatenate ((y_pred_out,y_pred_in))

    print ("Prédictions du REJECTOR sur les données de test de MNIST des classes attendues : Min, ", np.min(y_pred_in), " Max", np.max(y_pred_in), ' Moyenne : ', np.mean(y_pred_in))
    print ("Prédictions du REJECTOR sur les données de test de MNIST des classes non attendues : Min, ", np.min(y_pred_out), " Max", np.max(y_pred_out), ' Moyenne : ', np.mean(y_pred_out))
    print("Performances du REJECTOR sur toutes les données de test de MNIST :")
    print ("F1 : ", f1(Ytrue,Ypred), "Précision : ", precision(Ytrue,Ypred), "Recall rate : ", recall(Ytrue,Ypred))
    print(" Area Under Curve : " ,  auc(Ytrue, Ypred))
    print("")
    print ("Losses : DISCRIMINATOR ", d_losses," GENERATOR ", g_losses," CLASSIFIER ", c_losses," REJECTOR ", r_losses)
    
    #SAUVEGARDER ET MONTRER LES IMAGES GENEREES REGULIEREMENT
    if e == 1 or e % 5 == 0:
      plotGeneratedImages(e)
      saveModels(e)
    
    #MODIFICATION DU ALPHA DES RBF LAYERS
    b = rejector.layers[8].get_weights()
    alpha = b[0][0]
    alpha = alpha * Decrease_rate_alpha        
    b[0][0]=alpha
    rejector.layers[8].set_weights(b)
  plotLoss(e)

In [0]:
#@title ENTRAINEMENT DU GAN SEUL (télécharge toutes les epochs)
def train_GAN(X_train, Y_train, epochs=100, batchSize=128):
  #INITIALISATION...
  batchCount = int(X_train.shape[0] / batchSize)
  print ('Epochs:', epochs)
  print ('Batch size:', batchSize)
  print ('Batches per epoch:', batchCount)

  for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    dloss=0
    gloss=0
    for _ in tqdm(range(batchCount)):
      # DISCRIMINATOR
      noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
      generatedImages = generator.predict(noise)
      imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
      X = np.concatenate([imageBatch, generatedImages])
      yDis = np.zeros(2*batchSize)
      yDis[:batchSize] = 0.9
      dloss += discriminator.train_on_batch(X, yDis)

      # GENERATOR
      noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
      yGen = np.ones(batchSize)
      discriminator.trainable = False
      gloss += gan.train_on_batch(noise, yGen)
      
    # TESTS ET EVALUATIONS
    d_losses.append(dloss)
    g_losses.append(gloss)
    print ("Losses : DISCRIMINATOR ", d_losses[-1]," GENERATOR ", g_losses[-1])
    
    saveModels(e)
    downloadModels(e)
    plotLoss(e)

In [0]:
#@title ENTRAINEMENT DU REJECTOR SEUL (télécharge la dernière epoch) (WIP)
def train_rejector_on_bdd(X_train, bdd, batchSize=64):
  #INITIALISATION...
  if bdd == 1:
    genID = drive.ListFile({'title':'generator_e%d' % epoch}).GetList()[0]['id']
  batchCount = int(X_train.shape[0] / batchSize)
  print ('Epochs:', epochs)
  print ('Batch size:', batchSize)
  print ('Batches per epoch:', batchCount)

  for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    rloss=0
    for _ in tqdm(range(batchCount)):
      # DISCRIMINATOR
      noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
      generatedImages = generator.predict(noise)
      imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
      X = np.concatenate([imageBatch, generatedImages])
      yDis = np.zeros(2*batchSize)
      yDis[:batchSize] = 0.9
      dloss += discriminator.train_on_batch(X, yDis)

      # GENERATOR
      noise = np.random.normal(0, 1, size=[batchSize, randomVectorSize])
      yGen = np.ones(batchSize)
      discriminator.trainable = False
      gloss += gan.train_on_batch(noise, yGen)
      
    # TESTS ET EVALUATIONS
    d_losses.append(dloss)
    g_losses.append(gloss)
    print ("Loss : REJECTOR ", r_losses[-1])
    
    saveModels(e)
    downloadModels(e)
    plotLoss(e)
    
    #MODIFICATION DU ALPHA DES RBF LAYERS
    b = rejector.layers[8].get_weights()
    alpha = b[0][0]
    alpha = alpha * Decrease_rate_alpha        
    b[0][0] = alpha
    rejector.layers[8].set_weights(b)

In [0]:
#@title GENERATION DE LA BASE DE DONNEES (méthode 1.1/1.2)
def bdd_1_1(epochs, batchSize=64, batchNumber=50, qualityCheck='current', GANepochs=99, maxTry = 512, minTry = 64, shuffle = True):
  # QUALITYCHECK : 'current' ou 'final', passe de 1) à 2)
  bdd = np.zeros(epochs,batchSize*batchNumber,28,28,1)
  for i in range(epochs):
    length = 0
    final_length = batchSize*batchNumber
    epoch_bdd = np.zeros((final_length,28,28,1))
    generator, discriminator = uploadModels(i)
    if qualityCheck == 'final':
      discriminator = uploadModels(GANepochs)[1]
    while length != final_length:
      compute = max([min([final_length-length,maxTry]),minTry])
      noise = np.random.normal(0, 1, size=[compute, randomVectorSize])
      examples = generator.predict(noise)
      scores = discriminator.predict(examples)
      examples = examples[np.where(scores[:,0] < seuil_rejet_RejectionTraining)]
      for nb,i in enumerate(examples):
        if length < final_length:
          epoch_bdd[length,:] = i[:]
          length += 1
    if shuffle:
      np.random.shuffle(epoch_bdd)
    bdd[i,:] = epoch_bdd[:]
    with open("bdd_1_1_e"+str(i),'wb') as file:
      m=pickle.Pickler(file)
      m=pickle.dump(epoch_bdd,file)
    uploaded = drive.CreateFile({'title': 'bdd_1_1_e%d' % i})
    uploaded.SetContentFile('bdd_1_1_e%d' % i)
    uploaded.Upload()
  return bdd

In [0]:
#@title GENERATION DE LA BASE DE DONNEES (méthode 1.3/1.4)
def bdd_1_2(epochs, batchSize=64, batchNumber=50, qualityCheck='current', GANepochs=99, maxTry = 512, minTry = 64, shuffle = True):
  # QUALITYCHECK : 'current' ou 'final', passe de 1) à 2)
  bdd = np.zeros(epochs,batchSize*batchNumber,28,28,1)
  for i in range(epochs):
    length = 0
    if i == 0:
      final_length = batchSize*batchNumber
    else:
      final_length = batchSize*batchNumber/2
    epoch_bdd = np.zeros((batchSize*batchNumber,28,28,1))
    generator, discriminator = uploadModels(i)
    if qualityCheck == 'final':
      discriminator = uploadModels(GANepochs)[1]

    #PARTIE 1 : EXEMPLES DE L'EPOCH CORRESPONDANTE
    while length != final_length:
      compute = max([min([final_length-length,maxTry]),minTry])
      noise = np.random.normal(0, 1, size=[compute, randomVectorSize])
      examples = generator.predict(noise)
      scores = discriminator.predict(examples)
      examples = examples[np.where(scores[:,0] < seuil_rejet_RejectionTraining)]
      for nb,i in enumerate(examples):
        if length < final_length:
          epoch_bdd[length,:] = i[:]
          length += 1

    #PARTIE 2 : EXEMPLES DES EPOCHS PRECEDENTES
    numberPerEpoch = ceil((batchSize*batchNumber - final_length) / float(i))
    for j in range(i):
      generator, discriminator = uploadModels(j)
      if qualityCheck == 'final':
        discriminator = uploadModels(GANepochs)[1]
      while length <= final_length + numberPerEpoch*(j+1) and length != batchSize*batchNumber:
        compute = max([min([final_length-length,maxTry]),minTry])
        noise = np.random.normal(0, 1, size=[compute, randomVectorSize])
        examples = generator.predict(noise)
        scores = discriminator.predict(examples)
        examples = examples[np.where(scores[:,0] < seuil_rejet_RejectionTraining)]
        for nb,i in enumerate(examples):
          if length < batchSize*batchNumber:
            epoch_bdd[length,:] = i[:]
            length += 1

    #PARTIE 3 : ENREGISTREMENT
    if shuffle:
      np.random.shuffle(epoch_bdd)
    bdd[i,:] = epoch_bdd[:]
    with open("bdd_1_2_e"+str(i),'wb') as file:
      m=pickle.Pickler(file)
      m=pickle.dump(epoch_bdd,file)
    uploaded = drive.CreateFile({'title': 'bdd_1_2_e%d' % i})
    uploaded.SetContentFile('bdd_1_2_e%d' % i)
    uploaded.Upload()
  return bdd

In [0]:
#@title GENERATION DE LA BASE DE DONNEES (méthode 1.7)
def bdd_1_3(epochs, batchSize=64, batchNumber=50, qualityCheck='current', GANepochs=99, maxTry = 512, minTry = 64, shuffle = True):
  # QUALITYCHECK : 'current' ou 'final', passe de 1) à 2)
  bdd = np.zeros(epochs*batchSize*batchNumber,28,28,1)
  #repartition est telle que sum(repartition) = epochs*batchSize*batchNumber
  repartition = [epochs*batchSize*batchNumber/GANepochs for i in range(GANepochs)]
  while sum(repartition) != epochs*batchSize*batchNumber:
    repartition[len(repartition)//2]+=1
  length = 0

  #PARTIE 1 : RASSEMBLEMENT DES EXEMPLES
  for i in range(epochs):
    final_length = length + repartition[i]
    generator, discriminator = uploadModels(i)
    if qualityCheck == 'final':
      discriminator = uploadModels(GANepochs)[1]
    while length != final_length and length != batchSize*batchNumber:
      compute = max([min([final_length-length,maxTry]),minTry])
      noise = np.random.normal(0, 1, size=[compute, randomVectorSize])
      examples = generator.predict(noise)
      scores = discriminator.predict(examples)
      examples = examples[np.where(scores[:,0] < seuil_rejet_RejectionTraining)]
      for nb,i in enumerate(examples):
        if length < final_length:
          bdd[length,:] = i[:]
          length += 1
  
  #PARTIE 2 : TRI ET ENREGISTREMENT
  discriminator = uploadModels(GANepochs)[1]
  scores = discriminator.predict(bdd)
  scores_sorted = [(i,scores[i]) for i in range(scores.shape[0])]
  scores_sorted.sort(key = lambda x:x[1])
  order = [i[0] for i in scores_sorted]
  bdd = bdd[order,:]
  bdd_to_return = np.zeros(epochs,batchSize*batchNumber,28,28,1)
  for i in range(epochs):
    to_shuffle = bdd[i*batchSize*batchNumber:(i+1)*batchSize*batchNumber,:]
    if shuffle:
      np.random.shuffle(to_shuffle)
    bdd_to_return[i,:] = to_shuffle[:]
    with open("bdd_1_3_e"+str(i),'wb') as file:
      m=pickle.Pickler(file)
      m=pickle.dump(to_shuffle,file)
    uploaded = drive.CreateFile({'title': 'bdd_1_3_e%d' % i})
    uploaded.SetContentFile('bdd_1_3_e%d' % i)
    uploaded.Upload()
  return bdd_to_return

In [0]:
#@title GENERATION DE LA BASE DE DONNEES (méthode 2 avec répartition imposée) (WIP)
def bdd_2_1(epochs, repartition = None, batchSize=64, batchNumber=50, qualityCheck='current', GANepochs=99, maxTry = 512, minTry = 64, shuffle = True):
  bdd = np.zeros(epochs*batchSize*batchNumber,28,28,1)
  #repartition est telle que sum(repartition) = epochs*batchSize*batchNumber
  if repartition == None:
    repartition = [epochs*batchSize*batchNumber/GANepochs for i in range(GANepochs)]
    while sum(repartition) != epochs*batchSize*batchNumber:
      repartition[len(repartition)//2]+=1
  length = 0

  #PARTIE 1 : RASSEMBLEMENT DES EXEMPLES
  for i in range(epochs):
    final_length = length + repartition[i]
    generator, discriminator = uploadModels(i)
    if qualityCheck == 'final':
      discriminator = uploadModels(GANepochs)[1]
    while length != final_length and length != batchSize*batchNumber:
      compute = max([min([final_length-length,maxTry]),minTry])
      noise = np.random.normal(0, 1, size=[compute, randomVectorSize])
      examples = generator.predict(noise)
      scores = discriminator.predict(examples)
      examples = examples[np.where(scores[:,0] < seuil_rejet_RejectionTraining)]
      for nb,i in enumerate(examples):
        if length < final_length:
          bdd[length,:] = i[:]
          length += 1
  
  #PARTIE 2 : TRI ET ENREGISTREMENT
  discriminator = uploadModels(GANepochs)[1]
  scores = discriminator.predict(bdd)
  scores_sorted = [(i,scores[i]) for i in range(scores.shape[0])]
  scores_sorted.sort(key = lambda x:x[1])
  order = [i[0] for i in scores_sorted]
  bdd = bdd[order,:]
  bdd_to_return = np.zeros(epochs,batchSize*batchNumber,28,28,1)
  for i in range(epochs):
    to_shuffle = bdd[i*batchSize*batchNumber:(i+1)*batchSize*batchNumber,:]
    if shuffle:
      np.random.shuffle(to_shuffle)
    bdd_to_return[i,:] = to_shuffle[:]
    with open("bdd_2_e"+str(i),'wb') as file:
      m=pickle.Pickler(file)
      m=pickle.dump(to_shuffle,file)
    uploaded = drive.CreateFile({'title': 'bdd_2_e%d' % i})
    uploaded.SetContentFile('bdd_2_e%d' % i)
    uploaded.Upload()
  return bdd_to_return, weights

# Lancements

In [0]:
train_GAN(X_train, Y_train, epochs=100, batchSize=128)

  0%|          | 0/468 [00:00<?, ?it/s]

Epochs: 100
Batch size: 128
Batches per epoch: 468
--------------- Epoch 1 ---------------


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
100%|██████████| 468/468 [00:43<00:00, 10.80it/s]


Losses : DISCRIMINATOR  306.69124668836594  GENERATOR  388.0207463502884


  0%|          | 1/468 [00:00<00:55,  8.48it/s]

...

100%|██████████| 468/468 [00:40<00:00, 11.59it/s]


Losses : DISCRIMINATOR  297.0547565817833  GENERATOR  435.8218546509743


  0%|          | 1/468 [00:00<00:55,  8.44it/s]

--------------- Epoch 64 ---------------


 32%|███▏      | 148/468 [00:12<00:27, 11.48it/s]

# TESTS

In [0]:
#@title SEUILLAGES

"""Importer le modèle à 95-96%"""

model_95=load_model('10_classifier')

#entropie
seuils=np.concatenate((np.linspace(3.3,3.32,4),np.linspace(3.32,3.323,20)))
x=[]
y=[]
lim=10001
current=0
i=0
while i<len(seuils) and current<lim:
  print(str(i+1)+"/"+str(len(seuils)))
  point=test_model_entropie(model_95,seuils[i],x_test,y_test)
  x.append(point[0])
  y.append(point[1])
  i+=1
  current=point[0]

  
o=np.ones(len(x))
plt.plot(x,y,'b',label="entropie")
plt.plot(x,o,'r')

#max
seuils=np.concatenate((np.linspace(0,0.1,6),np.linspace(0.1,0.11,10),np.linspace(0.11,1,2)))
x=[]
y=[]
lim=10000
current=0
i=0
while i<len(seuils) and current<lim:
  print(str(i+1)+"/"+str(len(seuils)))
  point=test_model(model_95,seuils[i],x_test,y_test)
  x.append(point[0])
  y.append(point[1])
  print(point)
  i+=1
  current=point[0]


plt.plot(x,y,'g',label="max")
plt.xlabel("Nombre d'images rejetées")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# DERNIERS CHANGEMENTS / ETAT DES LIEUX

Ajout du code d'Artieres : TERMINE

Test du code d'Artieres : OK
- Entraînement : OK
- Tests : OK
- Autres fonctions : OK

Codes d'entraînement séparés : EN COURS
- GAN seul : A TESTER
- Création BDD : A FAIRE
- REJECTOR seul : A FAIRE
- CLASSIFIEUR seul : A ECRIRE (importer le modèle déjà prêt)

Nouvelles fonctions de choix des exemples : A VENIR

METHODE 1 : Curriculum learning

Plusieurs possibilités d'apprentissage progressif :

1) Chaque epoch d'apprentissage du REJECTOR est composée d'exemples générés par l'epoch correspondante du generator et filtrés par le DISCRIMINATOR correspondant

2) Même chose mais on filtre avec le DISCRIMINATOR final

3) Chaque epoch d'apprentissage du REJECTOR est composée d'exemples générés par l'epoch correspondante et d'exemples générés précédemment, filtrés par le DISCRIMINATOR correspondant

4) Même chose mais on filtre avec le DISCRIMINATOR final

[5) Pour chaque epoch, le nième batch est constitué d'images générées par la nième epoch du GAN, filtrées par le DISCRIMINATOR correspondant]

[6) Même chose mais on filtre avec le DISCRIMINATOR final]

7) On commence par générer une grande base de données avec toutes les epochs du GAN, que l'on ordonne grâce à leurs score par le DISCRIMINATOR final pour créer les batchs des différentes epochs

Priorité : METHODE 2 : Traditionnal learning

On commence par générer une base de données avec les différentes epochs du GAN, pour lesquelles on a les options suivantes :
 - filtrer par le DISCRIMINATOR correspondant ou le DISCRIMINATOR final
 - attribuer un poids grâce au score du DISCRIMINATOR final ou non
 - inclure des exemples de MNIST découpés ou non, que l'on peut tester avec le DISCRIMINATOR final ou non
 - mélanger la base de données avant de commencer ou non
 - imposer la répartition des exemples entre les différentes epochs du GAN ou laisser générer en fonction du pourcentage d'exemples pertinents à chaque epochs


Pour la base de données positive, on choisit évidemment les classes retenues de MNIST, et on choisit les éléments de chaque batch aléatoirement avec remise.

A AJOUTER :
 - utilisation de plusieurs GANs
 - autres méthodes génératives
 - changer les paramètres ou initialisations des mêmes GANs